In [ ]:
# Import das bibliotecas utilizadas no projeto

import pandas as pd
import numpy as np
import nltk

: 

In [21]:
# Import dos dados que serão analisados

df = pd.read_excel(r"data\dados_funks.xlsx", usecols=['Nº da ficha:', 'Título da música:', 'Canal:',
       'Nº de visualizações:', 'Nº de inscritos:', 'Data de publicação', 'MC:','DJ:', 'Duração:', 
       'Estado:', 'Cidade:', 'Observações:', 'Link:','Data de catalogação', 'Transcrição:'])

### Análise exploratória dos dados

In [22]:
df.head(3)

,Nº da ficha:,Título da música:,Canal:,Nº de visualizações:,Nº de inscritos:,Data de publicação,MC:,DJ:,Duração:,Estado:,Cidade:,Observações:,Link:,Data de catalogação,Transcrição:
0,1,Pela 1 vez juntos :: MC`S. Menor do Chapa e Ma...,Roda de Funk Original,240937.0,695998.0,2014-04-28 00:00:00,Menor do Chapa e Mazinho,NaN,11:01:00,Rio de Janeiro,Rio de Janeiro,•\tA MC Novinha é uma das poucas mulheres que ...,https://www.youtube.com/watch?v=Try7PPLOzKg,2017-11-12,1.\tMC Novinha: - Nós somos PCM porque temos d...
1,2,Mc sadrak Papo reto bonde dos 40 pedrinhas,lucas Sousa,95042.0,892.0,2014-08-01 00:00:00,Sadrak,NaN,02:50:00,Maranhão,São Luís,NaN,https://www.youtube.com/watch?v=O4QaVkn6zik E...,2017-12-19,Se for falar que é cv no jura nao pode mais\n\...
2,3,Bonde dos 40 :::: Mc Sadrak ((Sonho de Liberda...,Conexão B.40,3938.0,3400.0,2016-05-26 00:00:00,Sadrak,não informado,03:39:00,Maranhão,São Luís,•\t“Nós já nascemo pecando/ Tu já sabe como é ...,https://www.youtube.com/watch?v=XN2zGJ4SDKY E...,2017-12-19,Só soldado preparado no Jacaré\n\nChamada jorn...


In [23]:
df.shape

(120, 15)

In [24]:
df.columns

Index(['Nº da ficha:', 'Título da música:', 'Canal:', 'Nº de visualizações:',
       'Nº de inscritos:', 'Data de publicação', 'MC:', 'DJ:', 'Duração:',
       'Estado:', 'Cidade:', 'Observações:', 'Link:', 'Data de catalogação',
       'Transcrição:'],
      dtype='object')

In [26]:
df['MC:'].value_counts()

não informado                18
Moskito                       9
Sadrak                        9
Segal                         5
Dia JP                        4
                             ..
DD                            1
 Latrozinho                   1
Selinho e (?)                 1
Claudio e Ratinho             1
Menor do Chapa e Pedrinho     1
Name: MC:, Length: 64, dtype: int64

In [29]:
df['Estado:'].value_counts()

Maranhão               75
Rio de Janeiro         19
São Paulo               5
Paraíba                 3
Amazonas                3
Rio Grande do Sul       3
Ceará                   3
Bahia                   2
Santa Catarina          2
Interestadual           2
Rio Grande do Norte     1
Acre                    1
Mato Grosso             1
Name: Estado:, dtype: int64

In [31]:
df['Cidade:'].value_counts()

São Luís               71
Rio de Janeiro         18
São Paulo               6
Manaus                  3
Intercidades            3
João Pessoa             3
Salvador                2
Fortaleza               2
Florianópolis           2
Natal                   1
Porto Alegre            1
São José de Ribamar     1
Rio Branco              1
Caucaia                 1
Cuiabá                  1
Paço do Lumiar          1
Pindaré                 1
Niterói                 1
São Gonçalo             1
Name: Cidade:, dtype: int64

### Limpeza e processamento

In [32]:
df.columns

Index(['Nº da ficha:', 'Título da música:', 'Canal:', 'Nº de visualizações:',
       'Nº de inscritos:', 'Data de publicação', 'MC:', 'DJ:', 'Duração:',
       'Estado:', 'Cidade:', 'Observações:', 'Link:', 'Data de catalogação',
       'Transcrição:'],
      dtype='object')

In [33]:
# Convertendo todas as transcrições para letras minúsculas

df['Transcrição:'] = df['Transcrição:'].str.lower()

In [34]:
# Eliminação de espaços vazios

df['Transcrição:'] = df['Transcrição:'].str.strip()